In [ ]:
#@title ## Coursework: Train GPT2 & Generate samples

In [ ]:
import os
from IPython.display import clear_output
import ipywidgets

from google.colab import drive, files
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# get library requirements and supporting files from google drive
drive_path = '/content/drive/My Drive/MSc/Sem2_CC/'
os.chdir(drive_path)

In [ ]:
try:
  import gpt_2_simple as gpt2
except Exception as e:
  # if this is running the first time then download required libraries
  print(e)
  !pip install -r requirements.txt
  import gpt_2_simple as gpt2

  # There is a bug in jax, so uninstall this
  !pip uninstall -y jax jaxlib
  !pip install diffusers==0.11.1
  gpt2.download_gpt2(model_name="124M")

  clear_output()

In [ ]:
import nltk
nltk.download('punkt')

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tf.config.list_physical_devices()

In [ ]:
dir_ = 'train_content/'

In [ ]:
import helper_functions as help_

In [ ]:
# Getting all the content and tagging it for learning semantic embeddings
content = []
for f in os.listdir(dir_):
    path = dir_ + f
    text = open(path,'r').read().replace('\n','')
    content.append(text)
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(content)]
#
model = Doc2Vec(vector_size=100,
                min_count=3, epochs=20)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=80)
model.save("doc2vec_model.model")

document_vectors = [model.infer_vector(
                        word_tokenize(doc.lower())
                    )
                    for doc in content]

document_vectors = np.array(document_vectors)
avg_embeddings = np.average(document_vectors,axis=0)
np.save('avg_corpus_embeddings.npy', avg_embeddings)

In [ ]:
tf_session1 = gpt2.start_tf_sess()
gpt2.finetune(tf_session1, dataset=dir_, model_name='124M', steps=500,
              restore_from='latest', run_name='run1',
              checkpoint_dir='checkpoint/', batch_size=1,
              sample_length=128, print_every=10, sample_every=100, save_every=50
              )
gpt2.copy_checkpoint_to_gdrive(copy_folder=drive_path)

In [ ]:
with open('avg_corpus_embeddings.npy', 'rb') as f:
    avg_embeddings = np.load(f)

model = Doc2Vec.load('doc2vec_model.model')

In [ ]:
tf.compat.v1.reset_default_graph()
session0 = gpt2.start_tf_sess()
gpt2.load_gpt2(session0, checkpoint='model-500')

In [ ]:
gpt2.generate(session0, run_name='run1',
              prefix='<|startoftext|>',
              destination_path='story_samples.txt',
              nsamples=10,
              top_k=20, include_prefix=False
            )